# Something something big data on plane crashes woooo yeeeaaaaaaaaaah!!
                                            ##### Jupyter Notebook coded by Allison Ward, Rick Lataille, and Anthony Mansion

In [179]:
#Import the modules we need
import pandas as pd
import numpy as np

# And this is the big data that we will be using
df = pd.read_csv('Aviation_Data.csv', low_memory=False)

## Here, we will be dropping columns we see no need for, the information isn't relevant.

In [180]:
# Drop the columns we know that we don't need
dropped_columns = ['Schedule', 'Report.Status', 'Publication.Date']
df.drop(columns = dropped_columns, inplace=True)
print(f"{len(df)} items.")

90348 items.


## As for the columns we DO need, we filter some columns for the information we need.

In [181]:
# Convert date column to datetime, then filter event dates to include 2013 and later
df['Event.Date'] = pd.to_datetime(df['Event.Date'])
df_filtered = df.loc[df['Event.Date'] >= '2013-01-01']
print(f"{len(df_filtered)} items.")

15829 items.


In [182]:
# Filter aircraft categories for Airplanes only
df_filtered = df_filtered.loc[df_filtered['Aircraft.Category'] == 'Airplane']
print(f"{len(df_filtered)} items.")

13262 items.


In [183]:
# Exclude Amateur-built planes
df_filtered = df_filtered.loc[df_filtered['Amateur.Built'] != 'Yes']
print(f"{len(df_filtered)} items.")

11726 items.


In [184]:
# Include only events that happened in the United States or US Territories
allowed_countries = ['United States']
df_filtered = df_filtered.loc[df_filtered['Country'].isin(allowed_countries)]
print(f"{len(df_filtered)} items.")

9465 items.


In [185]:
# Exclude certain identified purposes as irrelevant to our stakeholder
allowed_purposes = ['Personal', np.nan, 'Business', 'Executive/corporate', \
                    'Positioning', 'Other Work Use', 'Ferry', 'Unknown', 'Public Aircraft - Federal', \
                   'Public Aircraft - State', 'Public Aircraft - Local', 'Public Aircraft', 'PUBS']
df_filtered = df_filtered.loc[df_filtered['Purpose.of.flight'].isin(allowed_purposes)]
print(f"{len(df_filtered)} items.")

7320 items.


In [186]:
df_filtered.to_csv('Filtered_Aviation_Data.csv', index=False)

In [187]:
#Converting latitude and longitude from Degrees, Minutes, and Seconds to Decimal Degrees

df_filtered.dropna(subset=['Latitude', 'Longitude'], inplace=True)

def convert_latitude(x):
    degrees = float(x[:2])
    minutes = float(x[2:4])
    seconds = float(x[4:6])
    return degrees + minutes/60 + seconds/3600

df_filtered["new_lats"] = df_filtered['Latitude'].map(convert_latitude)

def convert_longitude(x):
    degrees = float(x[:3])
    minutes = float(x[3:5])
    seconds = float(x[5:7])
    return -(degrees + minutes/60 + seconds/3600)

df_filtered["new_longs"] = df_filtered['Longitude'].map(convert_longitude)

In [188]:
# Separating the city and state from the Location column and creating a new column for the state (from https://www.kaggle.com/code/tetianalys605/aviation-accidents-python-analysis)

df_filtered['State'] = df_filtered['Location'].str.split(',').str[1]

In [189]:
#Getting the top 15 states with the most incidents in our data set
df_filtered['State'].value_counts(ascending = False).head(15)

State
CA    804
AK    751
TX    716
FL    688
AZ    313
CO    279
GA    271
WA    248
NC    213
ID    209
NY    196
PA    194
OH    187
OR    183
MI    179
Name: count, dtype: int64

In [190]:
# Taking a look at weather conditions
df_filtered['Weather.Condition'].value_counts()

Weather.Condition
VMC    7832
IMC     458
Unk      98
Name: count, dtype: int64

In [191]:
# Determining the states with the highest number of incidents under VMC conditions

state_VMC = df_filtered['State'].loc[df_filtered['Weather.Condition'] == 'VMC']
top_15_state_VMC = state_VMC.value_counts().head(15)
top_15_state_VMC

State
CA    727
AK    681
TX    640
FL    630
AZ    293
CO    247
GA    236
WA    223
ID    198
NC    187
PA    176
NY    172
OH    172
OR    165
MI    158
Name: count, dtype: int64

In [192]:
# Determining the states with the highest number of incidents under IMC conditions
state_IMC = df_filtered['State'].loc[df_filtered['Weather.Condition'] == 'IMC']
top_15_state_IMC = state_IMC.value_counts().head(15)
top_15_state_IMC

State
TX    39
AK    33
FL    30
CA    24
GA    23
NC    18
CO    16
TN    16
NY    15
IL    14
MN    13
MO    12
WI    12
VA    12
MI    12
Name: count, dtype: int64

In [197]:
# Looking at fatality counts per state
state_fatality = df_filtered['State'].loc[df_filtered['Injury.Severity'] == 'Fatal']
state_fatality_top_15 = state_fatality.value_counts().head(15)
state_fatality_top_15

State
CA    166
TX    113
FL    108
AK     76
GA     62
CO     45
AZ     37
NC     36
NY     35
WA     29
MI     29
UT     28
NM     28
OH     28
TN     28
Name: count, dtype: int64

In [199]:
# Looking at non-fatality counts per state
state_non_fatality = df_filtered['State'].loc[df_filtered['Injury.Severity'] != 'Fatal']
state_non_fatality_top_15 = state_non_fatality.value_counts().head(15)
state_non_fatality_top_15

State
AK    675
CA    638
TX    603
FL    580
AZ    276
CO    234
WA    219
GA    209
ID    186
NC    177
PA    175
NY    161
OH    159
OR    158
MI    150
Name: count, dtype: int64